<a href="https://colab.research.google.com/github/lkarjun/malayalam-language-model/blob/language-model/training_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset Loading

In [7]:
!pip install -qq dvc[gdrive]

!dvc get https://github.com/lkarjun/malayalam-language-model \
Datasets/


    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Enter verification code: 4/1AX4XfWiN4jIYvY9yEstRPTwg940SXCddQl9Gj5SsZRLcKsq4IVA0yjkzxGI
Authentication successful.


In [8]:
!unzip -q 'Datasets/*.zip' -d Datasets/


3 archives were successfully processed.


In [13]:
import pandas as pd
from pathlib import Path

DS_Path = Path("/content/Datasets")

csv_files = ["magazine_files.csv", 
             "wikitext_files.csv",
             "article_files.csv"
             ]
             
df = pd.concat([pd.read_csv(DS_Path/csv) for csv in csv_files])

In [41]:
sample = df['file_path'][:10].to_list()

with open(sample[0], "r") as file:
  sample_txt = file.read()

### Imports

In [1]:
!pip install -qq tokenizer transformers

     |████████████████████████████████| 77 kB 2.6 MB/s 
     |████████████████████████████████| 3.5 MB 9.2 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 6.8 MB 37.0 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 


In [237]:
from tokenizers import Tokenizer, pre_tokenizers, decoders
from tokenizers.models import Unigram, WordPiece
from tokenizers.trainers import UnigramTrainer, WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.pre_tokenizers import Digits
from tokenizers.normalizers import Strip

## Training Subword Tokenizer For Malayalm 

In [259]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Strip()
tokenizer.decoders = decoders.WordPiece()

trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [258]:
pre_tokenizer = pre_tokenizers.Sequence([
                                         Whitespace(),  
                                         Digits(individual_digits=False)
                                        ])


tokenizer.pre_tokenizer = pre_tokenizer

# training tokenizer
tokenizer.train(sample, trainer)

In [243]:
tokenizer.post_processor = TemplateProcessing(
                                single="[CLS] $A [SEP]",
                                pair="[CLS] $A [SEP] $B:1 [SEP]:1",
                                special_tokens=[
                                        ("[CLS]", tokenizer.token_to_id("[CLS]")),
                                        ("[SEP]", tokenizer.token_to_id("[SEP]")),
                                  ],
                              )

In [250]:
output = tokenizer.encode(sample_txt)
print(output.tokens)

['[CLS]', 'ഇന്ത്യയിലെ', 'ആദ്യത്തെ', 'വനിതാ', 'ഐ\u200cഎ\u200cഎസ്', 'ഉദ്യോഗസ്ഥയെ', 'കുറിച്ചറിയാമോ', '?', 'അവരുടെ', 'പേരാണ്', 'അന്ന', 'രാജം', 'മൽഹോത്ര', '.', 'അവർ', 'ഒരു', 'മലയാളി', 'കൂടിയാണ്', 'എന്നത്', 'നാം', 'മലയാളികൾക്ക്', 'അഭിമാനിക്കാനുള്ള', 'വകയാണ്', '.', 'കുഞ്ഞുകുട്ടി', 'പ്രാരാബ്ദങ്ങളുമായി', 'വീടുകളിൽ', 'ഒതുങ്ങി', 'കഴിഞ്ഞിരുന്ന', 'സ്ത്രീകൾക്കിടയിൽ', 'അന്ന', 'രാജം', 'മൽ\u200cഹോത്ര', 'വേറിട്ട്', 'നിന്നു', '.', 'സമൂഹത്തിൽ', 'നിലനിന്നിരുന്ന', 'അസമത്വത്തിനെതിരെ', ',', 'ലിംഗ', 'അനീതികൾക്കെതിരെ', 'പോരാടിയ', 'സ്ത്രീയാണ്', 'അവർ', ',', 'ഇന്ത്യയിലെ', 'ആദ്യത്തെ', 'വനിതാ', 'ഐ\u200cഎ\u200cഎസ്', 'ഉദ്യോഗസ്ഥ', '.', 'കേന്ദ്രസർക്കാരിൽ', 'സെക്രട്ടേറിയൽ', 'പദവി', 'വഹിച്ച', 'ആദ്യ', 'വനിത', 'കൂടിയായിരുന്നു', 'അവർ', '.', 'വളരെ', 'പ്രചോദനാത്മകമായ', 'അവരുടെ', 'കഥ', '.', '1927', 'ജൂലൈ', '17', '-', 'ന്', 'കേരളത്തിലെ', 'നിരണം', 'ഗ്രാമത്തിൽ', 'ജനിച്ച', 'അന്ന', ',', 'മദ്രാസ്', 'സർവകലാശാലയിൽ', 'നിന്ന്', 'ഇംഗ്ലീഷ്', 'സാഹിത്യത്തിൽ', 'ബിരുദാനന്തര', 'ബിരുദം', 'നേടി', '.', '1950', '-', 'ലാണ്', 'സിവിൽ', 'സർവീസ്', 'പരീക

In [251]:
print(output.ids)

[1, 2904, 1425, 2926, 3078, 7326, 7957, 28, 691, 5527, 319, 2789, 3111, 12, 467, 233, 3108, 2953, 1354, 3431, 7432, 7325, 7064, 12, 7120, 7837, 6713, 7547, 5235, 6783, 319, 2789, 7677, 4945, 741, 12, 5680, 3143, 8262, 10, 7711, 8013, 6443, 5404, 467, 10, 2904, 1425, 2926, 3078, 2862, 12, 8300, 7459, 7015, 7716, 581, 2925, 6453, 467, 12, 1180, 8260, 691, 499, 12, 6022, 7593, 1973, 11, 3423, 7385, 4402, 6897, 5715, 319, 10, 1954, 8162, 542, 7512, 6817, 6898, 3061, 2740, 12, 7307, 11, 2128, 2747, 1909, 6611, 6076, 319, 5796, 12, 4474, 8201, 233, 7920, 2747, 1909, 1870, 7944, 6332, 6514, 754, 5416, 5546, 12, 6724, 8297, 32, 5794, 1458, 6946, 12, 645, 10, 5489, 5443, 383, 319, 5003, 1434, 12, 1925, 8205, 1458, 1918, 2743, 12, 645, 10, 5585, 6610, 6502, 6597, 319, 6668, 12, 650, 1944, 7709, 7698, 6794, 1684, 12, 7306, 11, 95, 10, 6895, 319, 6032, 10, 233, 5401, 1396, 415, 3102, 5799, 12, 2903, 8256, 1909, 233, 5403, 5444, 7991, 989, 2511, 7794, 3102, 2833, 12, 645, 10, 8065, 5270, 319, 589, 

In [254]:
tokenizer.decode(output.ids, skip_special_tokens=True)

"ഇന്ത്യയിലെ ആദ്യത്തെ വനിതാ ഐ\u200cഎ\u200cഎസ് ഉദ്യോഗസ്ഥയെ കുറിച്ചറിയാമോ ? അവരുടെ പേരാണ് അന്ന രാജം മൽഹോത്ര . അവർ ഒരു മലയാളി കൂടിയാണ് എന്നത് നാം മലയാളികൾക്ക് അഭിമാനിക്കാനുള്ള വകയാണ് . കുഞ്ഞുകുട്ടി പ്രാരാബ്ദങ്ങളുമായി വീടുകളിൽ ഒതുങ്ങി കഴിഞ്ഞിരുന്ന സ്ത്രീകൾക്കിടയിൽ അന്ന രാജം മൽ\u200cഹോത്ര വേറിട്ട് നിന്നു . സമൂഹത്തിൽ നിലനിന്നിരുന്ന അസമത്വത്തിനെതിരെ , ലിംഗ അനീതികൾക്കെതിരെ പോരാടിയ സ്ത്രീയാണ് അവർ , ഇന്ത്യയിലെ ആദ്യത്തെ വനിതാ ഐ\u200cഎ\u200cഎസ് ഉദ്യോഗസ്ഥ . കേന്ദ്രസർക്കാരിൽ സെക്രട്ടേറിയൽ പദവി വഹിച്ച ആദ്യ വനിത കൂടിയായിരുന്നു അവർ . വളരെ പ്രചോദനാത്മകമായ അവരുടെ കഥ . 1927 ജൂലൈ 17 - ന് കേരളത്തിലെ നിരണം ഗ്രാമത്തിൽ ജനിച്ച അന്ന , മദ്രാസ് സർവകലാശാലയിൽ നിന്ന് ഇംഗ്ലീഷ് സാഹിത്യത്തിൽ ബിരുദാനന്തര ബിരുദം നേടി . 1950 - ലാണ് സിവിൽ സർവീസ് പരീക്ഷയ്ക്ക് ശ്രമിക്കാൻ അന്ന തീരുമാനിക്കുന്നത് . ഒരുപക്ഷേ അന്നത്തെക്കാലത്ത് ഒരു പെൺകുട്ടി സിവിൽ സർവീസ് സ്വപ്നം കാണുകയെന്നത്\u200c ആർക്കും ചിന്തിക്കാൻ പോലും സാധിക്കാത്ത കാര്യമാണ് . തീർത്തും വിപ്ലവകരമായ ആ തീരുമാനത്തെ കുടുംബം എതിർത്തു . എന്നാൽ , ചെയ്തേ പറ്റൂ എന്ന് അന്ന വാശി പിടിച്ചു . ഒ

In [255]:
check_unk = tokenizer.encode("Hello")
check_unk.tokens, check_unk.ids, tokenizer.id_to_token(0)

(['[CLS]', '[UNK]', '[SEP]'], [1, 0, 2], '[UNK]')

### Training Tokenizer

In [257]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Strip()
tokenizer.decoders = decoders.WordPiece()

trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence([
                                         Whitespace(),  
                                         Digits(individual_digits=False)
                                        ])


tokenizer.pre_tokenizer = pre_tokenizer

# training tokenizer
tokenizer.train(df['file_path'], trainer)

In [ ]:
tokenizer.post_processor = TemplateProcessing(
                                single="[CLS] $A [SEP]",
                                pair="[CLS] $A [SEP] $B:1 [SEP]:1",
                                special_tokens=[
                                        ("[CLS]", tokenizer.token_to_id("[CLS]")),
                                        ("[SEP]", tokenizer.token_to_id("[SEP]")),
                                  ],
                              )

In [ ]:
tokenizer.enable_padding(pad_id=3, pad_token="[PAD]")